In [944]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [945]:
dataset = pd.read_csv('./Boston.csv').drop(columns=['Unnamed: 0'])
dataset

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [946]:
X = np.array(dataset.drop(columns=['medv']))
y = np.array(dataset['medv'])

In [947]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [948]:
from torch.utils.data import Dataset, DataLoader

In [949]:
class MyDataset(Dataset):
    def __init__(self, data_x,data_y):
        self.data_x = data_x
        self.data_y = data_y

    def __len__(self):
        return len(self.data_x)

    def __getitem__(self, index):
        return self.data_x[index] ,self.data_y[index]

In [950]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [951]:
train_dataset = MyDataset(data_x=X_train,data_y=y_train)
test_dataset = MyDataset(data_x=X_test,data_y=y_test)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [952]:
import torch
import torch.nn as nn
import torch.optim as opti

In [953]:
class linear(nn.Module):
    def __init__(self, input_dim,output_dim):
        super(linear,self).__init__()
        self.fc1 = nn.Linear(input_dim, 50)
        self.fc2 = nn.Linear(50, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return x

In [954]:
model = linear(input_dim=13,output_dim=1)
model

linear(
  (fc1): Linear(in_features=13, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=1, bias=True)
  (relu): ReLU()
)

In [955]:
cirterion = nn.MSELoss()
optimizer = opti.SGD(model.parameters(),lr=0.001)

epochs = 50

for i in range(epochs):
    for x,y in train_dataloader:
        pred = model(x.to(torch.float32))
        loss = cirterion(pred,y.to(torch.float32))
        
        loss.backward() 
        optimizer.step()
        optimizer.zero_grad()
        print(loss.item())

704.3994750976562
767.856201171875
504.1495666503906
437.7149658203125
583.3297729492188
628.282470703125
441.7372741699219
455.56280517578125
561.1588134765625
583.033447265625
355.51885986328125
545.1845703125
593.3895874023438
418.9930419921875
440.2710266113281
332.8516540527344
406.70794677734375
465.4173889160156
305.0489807128906
404.7791748046875
372.9268798828125
193.92953491210938
218.96006774902344
126.86405181884766
193.974609375
103.4205093383789
198.11546325683594
112.78926849365234
174.0320281982422
154.21925354003906
66.39737701416016
110.01890563964844
71.75143432617188
84.68102264404297
133.9278564453125
63.57853317260742
159.97503662109375
70.13613891601562
133.1743927001953
114.60453796386719
131.69778442382812
108.68926239013672
104.0973129272461
71.59490966796875
102.03761291503906
44.48917007446289
72.2276611328125
92.71974182128906
47.3906364440918
111.19661712646484
137.60372924804688
78.07467651367188
89.80230712890625
94.92050170898438
78.43073272705078
87.32

/opt/anaconda3/envs/env_imageProcessing/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/anaconda3/envs/env_imageProcessing/lib/python3.12/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [956]:
model.eval()

all_pred = []
all_y = []

with torch.no_grad():
        for x,y in test_dataloader:
                pred = model(x.to(torch.float32))       
                all_pred.append(pred)
                all_y.append(y)

In [957]:
all_pred = torch.cat(all_pred, dim=0).numpy()
all_y = torch.cat(all_y, dim=0).numpy()

In [958]:
all_pred

array([[23.911465],
       [26.705835],
       [21.191017],
       [26.919321],
       [24.078594],
       [24.224783],
       [24.996855],
       [25.429739],
       [23.989317],
       [24.483618],
       [24.338514],
       [22.817137],
       [26.321291],
       [25.02718 ],
       [23.863054],
       [23.732319],
       [21.126976],
       [25.242905],
       [26.020067],
       [24.850792],
       [24.116709],
       [23.95912 ],
       [26.542233],
       [26.03971 ],
       [21.161097],
       [24.482876],
       [24.787062],
       [22.737595],
       [22.079554],
       [25.743612],
       [25.894676],
       [23.54133 ],
       [23.4107  ],
       [22.764982],
       [19.48038 ],
       [25.387575],
       [25.075575],
       [23.828278],
       [23.077257],
       [22.287245],
       [24.355078],
       [26.302984],
       [23.6971  ],
       [24.229652],
       [23.419828],
       [23.329298],
       [24.834545],
       [23.441488],
       [22.79984 ],
       [25.6261  ],


In [959]:
all_y

array([24.8, 23. , 17.2, 13.6, 18.5, 17.2, 50. , 48.5, 24.4, 16.8, 19.4,
       23.4, 35.4, 21.2, 17.2, 19.5,  9.6, 26.5,  5. , 19.1, 32.4, 19.7,
       10.9, 16.1, 14.9, 13.8, 20.9, 23.1, 27.5, 23.7, 24.1, 23.9, 20.3,
       20. , 17.9, 18.8, 18.5, 10.8, 24.2, 23.2, 21.5, 14.1, 20.8, 18.9,
       29. , 23.6, 21.7, 24. , 22.8, 25. , 20. , 31.5, 29.1, 20.2,  9.7,
       22.5, 25. , 35.4, 15.1, 20. , 25. , 12.7,  7. , 19.9, 18.4, 34.7,
       23.8, 15.2, 13.1, 15. , 20.8, 20.1, 19.4, 17.4, 10.2, 19.6, 29.8,
       14. , 16.4, 10.4,  7. , 28.2, 50. , 14.2, 28.5, 18.9, 13.1, 19.6,
       21.2, 18.3, 20.9, 25.2, 50. , 22.4, 12.7, 30.8, 43.5, 17.8, 17.1,
       22.6, 21.4, 20.6])

In [960]:
print('MAE =', mean_absolute_error(all_y,all_pred))
print('MSE =', mean_squared_error(all_y,all_pred))
print('RMSE =', np.sqrt(mean_squared_error(all_y,all_pred)))

MAE = 6.973293136147892
MSE = 83.09888804699409
RMSE = 9.11585915023889
